In [2]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


d:\python371\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Для обучения векторных представлений необходимо большое количество текста. Чем больше текста, тем лучше предтавления получатся.  
Возьмем ~7к новостных статей. Это все ещё маленький корпус, но для обучения он подходит (на нем можно достаточно быстро попробовать разные методы). 

In [7]:
data_rt = pd.read_csv('news_texts.csv')

In [8]:
%%time
data_rt['content_norm'] = data_rt['content'].apply(normalize)

AttributeError: 'float' object has no attribute 'lower'

In [10]:
data_rt.dropna(inplace=True)

## Матричные разложения

Попробуем сначала матричные разложения. В SVD и в NMF одна из получаемых матриц имеет размерность (количество слов, количесто "тем"). Вектора из этих матриц и будут искомыми эбмедингами.

In [37]:
N = len(data_rt['content_norm'])

### Векторные представления в настоящей задаче

Все вышеперечисленое относится к intrinsic (внутренним) метрикам. Есть также много других схожих (аналогии, корреляция с оценками людей и т.д). Но эти метрики не всегда показывают какой из методов сработает в реальной задаче. Поэтому при выборе методов и подборе параметров лучше ориентироваться на оценки качества решаемой задачи.

Проверим как все эти методы работают на задаче определения парафразов (предложений, которые выражают одно и то же, но не равны друг другу).

Данные взяты вот отсюда: http://paraphraser.ru/

Коллекция состоит из пар предложения (заголвков статей) и метки класса (-1,0,1). -1 не парафраз, 1 - парафраз, 0 - что-то непонятное.

## ==========

In [11]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [12]:
%%time
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

Wall time: 34.2 s


In [13]:
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


Преобразуйте тексты в векторы в каждой паре 4 методами  - SVD, NMF, Word2Vec, Fastext.

 Для SVD и NMF сделайте две пары векторов - через TfidfVectorizer и CountVectorizer. 
 
 Обучать модели я буду на корпусе new_texts.csv с семинара

In [14]:
c_vec = CountVectorizer()
tfidf_vec = TfidfVectorizer()

c_vec2 = CountVectorizer()
tfidf_vec2 = TfidfVectorizer()

In [15]:
%%time
cv1 = c_vec.fit(data_rt['content_norm'])
tfidf1 = tfidf_vec.fit(data_rt['content_norm'])

Wall time: 21.5 s


In [16]:
%%time
cv2 = c_vec2.fit(data_rt['content'])
tfidf2 = tfidf_vec2.fit(data_rt['content'])

Wall time: 30.7 s


In [17]:
# transform
cv_data_n = cv1.transform(data_rt['content_norm'])
tfidf_data_n = tfidf1.transform(data_rt['content_norm'])
cv_data = cv2.transform(data_rt['content'])
tfidf_data = tfidf2.transform(data_rt['content'])

Теперь зафиттим модели на данных

In [18]:
%%time
svd_tfidf = TruncatedSVD(100)
svd_tfidf.fit(tfidf_data_n)

Wall time: 22.4 s


In [19]:
%%time
nmf_tfidf = NMF(100)
nmf_tfidf.fit(tfidf_data_n)

Wall time: 8min 33s


In [20]:
%%time
svd_cv = TruncatedSVD(100)
svd_cv.fit(cv_data_n)

Wall time: 19 s


In [21]:
%%time
nmf_cv = NMF(100)
nmf_cv.fit(cv_data_n)

Wall time: 15min 13s


Для word2vec сделайте две пары векторов - с взвешиванием по tfidf и без.

In [22]:
%%time
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=100, sg=1)

Wall time: 2min 31s


Для Fastext постройте две модели - без нормализации и с нормализацией

In [23]:
%%time
fast_text_norm = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=100, min_n=4, max_n=8)

Wall time: 3min 52s


In [24]:
%%time
corpus = [text.split() for text in data_rt['content'].apply(tokenize)]
fast_text = gensim.models.FastText(corpus, size=100, min_n=4, max_n=8)

Wall time: 4min 39s


In [25]:
from gensim import matutils
def cosine(x, y):
    x_norm = gensim.matutils.unitvec(np.array(x))
    y_norm = gensim.matutils.unitvec(np.array(y))
    return np.dot(x_norm, y_norm)

In [44]:
from collections import defaultdict, Counter
def inverted_index(texts) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    index = defaultdict(list)
    i = 0
    for text in texts:
        for word in text:
            if i not in index[word]:
                index[word].append(i)
        i += 1
    inv = {}
    for word in index:
        inv[word] = len(index[word])
        
    return inv

In [39]:
N = len(data)
N

7227

In [54]:
from math import log
def get_embedding(text, model, dim, N=0, inv_ind=False):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model.wv[word]
            if inv_idx:
                vectors[i] = v*(words[word]/total)*log(N/inv_idx[word])
            else:
                vectors[i] = v
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

 Между векторами каждой пары вычислите косинусную близость (получится 10 чисел для каждой пары текстов). 

In [28]:
%%time
X1 = svd_tfidf.transform(tfidf1.transform(data['text_1_norm']))
X2 = svd_tfidf.transform(tfidf1.transform(data['text_2_norm']))
sim_svd_tf = [cosine(x, y) for x, y in zip(X1, X2)]

d:\python371\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Wall time: 1.67 s


In [29]:
X1 = nmf_tfidf.transform(tfidf1.transform(data['text_1_norm']))
X2 = nmf_tfidf.transform(tfidf1.transform(data['text_2_norm']))
sim_nmf_tf = [cosine(x, y) for x, y in zip(X1, X2)]

d:\python371\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [32]:
X1 = svd_cv.transform(cv1.transform(data['text_1_norm']))
X2 = svd_cv.transform(cv1.transform(data['text_2_norm']))
sim_svd_cv = [cosine(x, y) for x, y in zip(X1, X2)]

In [31]:
X1 = nmf_cv.transform(cv1.transform(data['text_1_norm']))
X2 = nmf_cv.transform(cv1.transform(data['text_2_norm']))
sim_nmf_cv = [cosine(x, y) for x, y in zip(X1, X2)]

d:\python371\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [55]:
both_texts = np.concatenate([data['text_1_norm'], data['text_2_norm']], axis=0)
inv_idx = inverted_index(both_texts)

In [52]:
w2v.most_similar(positive=['обстрел'])

d:\python371\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('миномётный', 0.8455487489700317),
 ('обстреливать', 0.7861403226852417),
 ('обстрелять', 0.7761354446411133),
 ('артобстрел', 0.7750434875488281),
 ('макеевка', 0.7652963399887085),
 ('авдеевка', 0.7603738307952881),
 ('дебальцево', 0.7565488815307617),
 ('авианалёт', 0.7474933862686157),
 ('ясиноватая', 0.7461389899253845),
 ('боестолкновение', 0.7442883253097534)]

In [68]:
dim = 100
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, N, inv_ind=inv_idx)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, N, inv_ind=inv_idx)

d:\python371\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [78]:
sim_w2v_weighted = [cosine(x, y) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]

In [70]:
dim = 100
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, N)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, N)

d:\python371\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [71]:
sim_w2v = [cosine(x, y) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]

In [72]:
dim = 100
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text_norm, dim, N)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text_norm, dim, N)

d:\python371\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [73]:
sim_ft = [cosine(x, y) for x, y in zip(X_text_1_ft, X_text_2_ft)]

In [74]:
dim = 100
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text_norm, dim, N, inv_ind=inv_idx)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text_norm, dim, N, inv_ind=inv_idx)

d:\python371\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [75]:
sim_ft_weighted = [cosine(x, y) for x, y in zip(X_text_1_ft, X_text_2_ft)]

Объединим векторы

In [80]:
new_x = pd.DataFrame({1: sim_ft_weighted, 2: sim_ft, 3: sim_w2v, 4: sim_w2v_weighted, 5: sim_nmf_cv, 6: sim_svd_cv, 7: sim_nmf_tf, 8: sim_svd_tf})

In [81]:
y = data['label'].values
print(y.shape)

(7227,)


In [83]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score

In [85]:
log_reg = LogisticRegressionCV(cv=5, scoring='f1_micro', class_weight='balanced')

In [86]:
cross_val_score(log_reg, new_x, y, cv=5, scoring='f1_micro', n_jobs=-1).mean()

0.4366828333839955

In [88]:
forest = RandomForestClassifier(n_estimators=600, criterion='entropy', n_jobs=-1, class_weight='balanced')

In [89]:
cross_val_score(forest, new_x, y, scoring='f1_micro', cv=5, n_jobs=-1).mean()

0.49948445207121905

Теперь поиграем немного с параметрами.

In [90]:
%%time
nmf_cv = NMF(200)   # было 100
nmf_cv.fit(cv_data_n)

Wall time: 40min 31s


In [91]:
X1 = nmf_cv.transform(cv1.transform(data['text_1_norm']))
X2 = nmf_cv.transform(cv1.transform(data['text_2_norm']))
sim_nmf_cv = [cosine(x, y) for x, y in zip(X1, X2)]

d:\python371\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [92]:
%%time
svd_cv = TruncatedSVD(200)  # было 100
svd_cv.fit(cv_data_n)

Wall time: 24 s


In [93]:
X1 = svd_cv.transform(cv1.transform(data['text_1_norm']))
X2 = svd_cv.transform(cv1.transform(data['text_2_norm']))
sim_svd_cv = [cosine(x, y) for x, y in zip(X1, X2)]

In [98]:
new_x_2 = pd.DataFrame({1: sim_nmf_cv, 2: sim_svd_cv})

In [99]:
forest = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=-1, class_weight='balanced')  # было 600 эстиматоров

In [100]:
cross_val_score(forest, new_x_2, y, scoring='f1_micro', cv=5, n_jobs=-1).mean()

0.47181042543880125

## ==========